In [ ]:
import numpy as np
from skimage import color, data, restoration, img_as_float, io
from scipy.signal import convolve2d as conv2

from skimage.measure import compare_psnr
from functools import partial
from scipy.optimize import minimize
from scipy.optimize import OptimizeResult

from util import *
from richardson_lucy import *

In [ ]:
def fun(x):
    return np.sum((x-np.array([1,2]))**2)

In [ ]:
x0 = np.array([-1, -1])
xx = np.array([1, 2])

In [ ]:
fun(xx), fun(x0)

def minimize_grad(fun, x0, grad=None, alpha=3, ftol = 1e-9, xtol=1e-3, maxit=50, disp=False):
    def gradient(x, step=1):
        """suppose x is of size 2"""
        df = np.zeros(x.shape)
        for coord in range(x.size):
            dx = np.zeros(x.shape)
            dx[coord] = step
            df[coord] = (fun(x + dx) - fun(x - dx)) / (2 * step)
        return df
    def gradient2(x):
        fval = np.zeros((3,3))
        for dy in range(-1, 2):
            for dx in range(-1, 2):
                if not(dx == 0 and dy == 0):
                    fval[dy+1, dx+1] = fun(x+np.array([dx, dy]))
        sobel = np.array([[-1, 0, 1],
                          [-2, 0, 2],
                          [-1, 0, 1]])
        scale = np.sum(np.abs(sobel))
        dfdy = np.sum(fval * sobel.T)
        dfdx = np.sum(fval * sobel)
        G = np.array([dfdx, dfdy])
        return G# / np.sum(np.abs(G))
    
    if grad is None:
        grad = gradient2
    
    fin = False
    x = x0
    prev_f = fun(x)
    iterations = 0
    nfev = 0
    
    #gradient loop
    while not fin:
        iterations += 1
        df = grad(x)
        nfev += 8
        
        if disp:
            print(x, alpha, df)
        # optimal loop
        fin2 = False
        lamb = alpha
        while abs(lamb) > xtol:
            x1 = x - lamb*df
            cur_f = fun(x1)
            nfev += 1
            if cur_f < prev_f:
                x = x1
                prev_f= cur_f
            else:
                lamb *= -1/2
        #alpha *= .7
        cur_f = fun(x)
        if iterations == maxit or np.sum(np.abs(df))<ftol:
            fin = True
        prev_f = cur_f
    return OptimizeResult(x=x, nit=iterations, nfev=nfev)

In [ ]:
def fun2(x):
    return (x[0]-5)**2 + 2*(x[1]-10)**2

In [ ]:
minimize_grad(fun2, np.array([0,0]), alpha=1, disp=True, maxit=100)

In [ ]:
#res = minimize_grad(fun, x0, iter=100)

In [ ]:
#res['x']

In [ ]:
astro = color.rgb2gray(data.astronaut())
liftingbody = img_as_float(io.imread('liftingbody.png'))
iterations = 30

In [ ]:
sz = 20
deg = 5
points_real = [(deg-1)*sz//deg-1, sz//deg, sz-1, sz-1]
#points_real = [sz, 0, sz, sz]
psf_bezier = bezier_psf2(points_real)
plt.imshow(psf_bezier, 'gray')
plt.show()

In [ ]:
taper_psf = gkern2(41, 7)
plt.imshow(taper_psf, 'gray')
plt.show()

In [ ]:
lifting_bezier_blurred = convolve2d(liftingbody, psf_bezier, 'same', 'symm')
#lifting_bezier_blurred = edgetaper(lifting_bezier_blurred, taper_psf)
plt.imshow(lifting_bezier_blurred, 'gray')
plt.show()

In [ ]:
from skimage.restoration import richardson_lucy

In [ ]:
deconv = richardson_lucy(lifting_bezier_blurred, psf_bezier, iterations=40)#, dampar=s_n, useFFT=False
#plt.imshow(deconv['image'], 'gray')
plt.imshow(deconv, 'gray')
plt.show()

In [ ]:
s_n, S_find = find_noise(lifting_bezier_blurred)
s_n

In [ ]:
deconv = richardson_lucy_matlab(lifting_bezier_blurred, psf_bezier, iterations=30, dampar=10*s_n, useFFT=True)
plt.imshow(deconv['image'], 'gray')
#plt.imshow(deconv, 'gray')
plt.show()

In [ ]:
b_clip = 3
N,M = lifting_bezier_blurred.shape
# ( ifft2 (100* log (1+ abs ( fft2 ( I ) ) ) ) ) ;
K = np.fft.ifftn(100*np.log(1+np.abs(np.fft.fftn(lifting_bezier_blurred))))
K_shift = np.fft.fftshift(K)

nn = np.argmin(np.real(K_shift))
n, m, = K_shift.shape
x0 = np.array([nn // n - n//2, nn % n - m//2])
print(x0)

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(np.clip(np.real(K_shift),-1,1), 'gray')#, vmin=noisy.min(), vmax=noisy.max())
plt.plot(n//2 + x0[0], m//2 + x0[1], 'ro')
plt.title('Кепстр изображения с выделенным минмиумом')
plt.savefig('kepstr.png')
plt.show()

In [ ]:
x0 = np.concatenate((x0/3, x0))
print(x0)

In [ ]:
def funcToMinimizeCurved(xy, I_blurred, crop=100, *args, **kwargs):
    psf = bezier_psf2(xy, n=100)
    restored = richardson_lucy_matlab(I_blurred, psf, *args, **kwargs)
    I_restored = restored['image']
    df = convolve2d(I_restored, psf, 'same', 'symm') - I_blurred
    return np.mean(np.square(df[crop:-crop, crop:-crop]))
#partial(funcToMinimizeCurved, I_blurred=lifting_blurred, iterations=iterations, eps=1e-5, dampar=s_n, useFFT=True)([0, 10, 10, 10])

In [ ]:
def funcToMinimizeCurved2(xy, I_blurred, xy2, crop=100, *args, **kwargs):
    psf = bezier_psf2(np.concatenate((np.array(xy), np.array(xy2))), n=100)
    restored = richardson_lucy_matlab(I_blurred, psf, *args, **kwargs)
    I_restored = restored['image']
    df = convolve2d(I_restored, psf, 'same', 'symm') - I_blurred
    return np.mean(np.square(df[crop:-crop, crop:-crop]))

In [ ]:
def funcToMinimizeCurved3(xy, I_blurred, xy2, crop=100, *args, **kwargs):
    psf = bezier_psf2(np.concatenate((np.array(xy), np.array(xy2))), n=100)
    #restored = richardson_lucy_matlab(I_blurred, psf, *args, **kwargs)
    #I_restored = restored['image']
    I_restored = richardson_lucy(I_blurred, psf, *args, **kwargs)
    df = convolve2d(I_restored, psf, 'same', 'symm') - I_blurred
    return np.mean(np.square(df[crop:-crop, crop:-crop]))

In [ ]:
nn = np.argmin(np.real(K_shift))
n, m, = K_shift.shape
x0_2 = np.array([nn // n - n//2, nn % n - m//2])
x0_1 = np.array([nn // n - n//2, nn % n - m//2]) /3
print(x0_1, x0_2)

In [ ]:
res_my_grad = minimize_grad(partial(funcToMinimizeCurved2, I_blurred=lifting_bezier_blurred, xy2=x0_2, iterations=iterations, eps=1e-5, dampar=10*s_n, useFFT=True), x0=x0_1, disp=True, alpha=1000)
print(res_my_grad)
#res_my_

In [ ]:
res_my_grad2 = minimize_grad(partial(funcToMinimizeCurved2, I_blurred=lifting_bezier_blurred, xy2=x0_2, iterations=iterations, eps=1e-5, dampar=10*s_n, useFFT=True), x0=np.array([20,0]), disp=True, alpha=0.1)
print(res_my_grad)
#res_my_

In [ ]:
res_my_grad3 = minimize_grad(partial(funcToMinimizeCurved3, I_blurred=lifting_bezier_blurred, xy2=x0_2, iterations=iterations), x0=x0_1, disp=True, alpha=10000)
print(res_my_grad)
#res_my_

In [ ]:
res_my_grad2 = OptimizeResult(x=np.concatenate((res_my_grad['x'], x0_2)))

In [ ]:
deconv_orig = richardson_lucy_matlab(lifting_bezier_blurred, psf_bezier, iterations=iterations, dampar=s_n, useFFT=False)
psnr_orig = compare_psnr(liftingbody, deconv_orig['image'])

psf_init = bezier_psf2(x0)
deconv_init = richardson_lucy_matlab(lifting_bezier_blurred, psf_init, iterations=iterations, dampar=s_n, useFFT=False)
psnr_init = compare_psnr(liftingbody, deconv_init['image'])

points_found = res_my_grad2['x']
psf_found = bezier_psf2(points_found)
deconv_found = richardson_lucy_matlab(lifting_bezier_blurred, psf_found, iterations=iterations, dampar=s_n, useFFT=False)
psnr_found = compare_psnr(liftingbody, deconv_found['image'])

show_results(deconv_orig['image'], deconv_init['image'], deconv_found['image'],
             titles=['Restored with true psf\nPSNR={0}\ncoord={1}'.format(psnr_orig, points_real), 
                     'With initial approxiamtion\nPSNR={0}\ncoord={1}'.format(psnr_init, x0),
                     'Minimized error\nPSNR={0}\ncoord={1}'.format(psnr_found, res_my_grad2['x'])])